In [2]:
import pandas as pd
# import re
from datetime import datetime
import string

In [3]:
# Correção para linhas que começam com múltiplos espaços

def correctSplit(x):
    splitted_str = list(x)
    if splitted_str[0] == '|':
        splitted_str[0] = ''
    return ''.join(splitted_str)

# Função para separar colunas do df importado

def separar_colunas_df(df):
    # lista alfabetica e.g ['a','b','c'...]
    lista_alfabeto = list(string.ascii_lowercase)
    # Pega a primeira coluna do df (que tem todos os dados) e substitui onde possui mais de um espaço por | 
    df.iloc[:,0] = df.iloc[:,0].replace({'( ){2,} ':'|'},regex=True)
    # corrige a substituição para linhas que começam com espaços
    df.iloc[:,0] = df.iloc[:,0].apply(lambda x: correctSplit(x))
    # corta a partir do separador '|' e define cada pedaço para uma coluna ['a','b'..]
    for i in range (len(df.iloc[:,0][0].split('|'))):
        df[lista_alfabeto[i]] = df.iloc[:,0].apply(lambda x: x.split('|')[i])
    # deleta a coluna que originou as demais colunas    
    df.drop(df.columns[0], axis = 1, inplace = True)
    return df

# veic_2019_02.csv_amostra

In [4]:
colnames = ['index_df', 'x', 'y', 'data-hora', 'data', 'dia', 'mes', 
            'ano', 'tempo', 'hora', 'a',
           'b', 'c', 'd', 'e', 'f', 'placa', 'h', 
            'i', 'j', 'k', 'l' ]

df_veic = pd.read_csv('veic_2019_02.csv_amostra.txt', sep=",", names=colnames)
df_veic.head(4)

FileNotFoundError: [Errno 2] No such file or directory: 'veic_2019_02.csv_amostra.txt'

# movimento_OCR_JAN_2019.txt_amostra.txt

In [ ]:
# Nome para coluna inicial
colnames = ['x']

# Usando separador '|' apenas para conseguir importar e depois tentar tratar
df_ocr_raw = pd.read_csv('movimento_OCR_JAN_2019.txt_amostra.txt', sep='|', names = colnames)

df_ocr_split = separar_colunas_df(df_ocr_raw.copy())
df_ocr_split.head()

# Tratando OCR

In [ ]:
def get_datahora(string):
    date = string[38:52]
    try: 
        date = datetime.strptime(date, "%Y%m%d%H%M%S")
    except:
        date = 'erro'
    return date

def get_velocidade(string):
    vel = string[92:95]
    try:
        vel = int(vel)
    except:
        vel = 'erro'
    return vel

def get_placa(string):
    placa = string[0:7]
    return placa.strip()

df_ocr = pd.DataFrame()
df_ocr['CODCET'] = df_ocr_raw['x'].apply(lambda x: x[108:118])
df_ocr['DATAHORA'] = df_ocr_raw['x'].apply(lambda x: get_datahora(x))
df_ocr['PLACA'] = df_ocr_raw['x'].apply(lambda x: get_placa(x))
df_ocr['VELOCIDADE'] = df_ocr_raw['x'].apply(lambda x: get_velocidade(x))
df_ocr['CGT'] = df_ocr_raw['x'].apply(lambda x: x[108:111])
df_ocr['LATITUDE'] = df_ocr_raw['x'].apply(lambda x: x[216:225])
df_ocr['LONGITUDE'] = df_ocr_raw['x'].apply(lambda x: x[225:234])

# Salvando em csv
df_ocr.to_csv('amostra_movimento_tratada.csv', sep=';', index=False)
df_ocr.sample(5)

# SCP_INTVIA_0182018_201901_amostra.txt

In [5]:
# Nome para colunas desconhecidas
colnames = ['x']
# Usando separador '|' apenas para conseguir importar e depois tentar tratar
df_intvia_raw = pd.read_csv('SCP_INTVIA_0182018_201901_amostra.txt', sep='|', names = colnames)

# separando as colunas
df_intvia_split = separar_colunas_df(df_intvia_raw)

df_intvia_split.head()

FileNotFoundError: [Errno 2] No such file or directory: 'SCP_INTVIA_0182018_201901_amostra.txt'

# SCP_RADAR_RIO_SITRAN_0232015_201903_amostra.txt

In [6]:
# Nome para colunas desconhecidas
colnames = ['x']
# Usando separador '|' apenas para conseguir importar e depois tentar tratar
df_sitran_raw = pd.read_csv('SCP_RADAR_RIO_SITRAN_0232015_201903_amostra.txt', sep='|', names = colnames)
            
# Separando as colunas
df_sitran_split = separar_colunas_df(df_sitran_raw)

df_sitran_split.head()

FileNotFoundError: [Errno 2] No such file or directory: 'SCP_RADAR_RIO_SITRAN_0232015_201903_amostra.txt'

# SCP_TRANSCARIOCA_0222016_201903_amostra.txt

In [7]:
# Nome para colunas desconhecidas
colnames = ['x']
# Usando separador '|' apenas para conseguir importar e depois tentar tratar
df_transcarioca_raw = pd.read_csv('SCP_TRANSCARIOCA_0222016_201903_amostra.txt', sep='|', names = colnames)
            
# Separando as colunas
df_transcarioca_split = separar_colunas_df(df_transcarioca_raw.copy())
df_transcarioca_split.head(5)

FileNotFoundError: [Errno 2] No such file or directory: 'SCP_TRANSCARIOCA_0222016_201903_amostra.txt'

# Tratamento SCP_TRANSCARIOCA

In [8]:
def get_datahora(string):
    date = string[78:92]
    try: 
        date = datetime.strptime(date, "%d%m%Y%H%M%S")
    except:
        date = 'erro'
    return date

def get_placa(string):
    placa = string[:7]
    if placa[0].isalpha():
        pass
    else:
        placa = ''
    return placa

def get_velocidade(string):
    vel = string[92:94]
    try:
        vel = int(vel)
    except:
        vel = 'erro'
    return vel

def get_latitude(string):
    lat = string[216:219]+"."+string[220:225]
    return lat

def get_longitude(string):
    lon = string[225:228]+"."+string[229:]
    return lon

df_transcarioca = pd.DataFrame()
df_transcarioca['CODCET'] = df_transcarioca_raw['x'].apply(lambda x:x[108:118])
df_transcarioca['DATAHORA'] = df_transcarioca_raw['x'].apply(lambda x: get_datahora(x))
df_transcarioca['PLACA'] = df_transcarioca_raw['x'].apply(lambda x: get_placa(x))
df_transcarioca['VELOCIDADE'] = df_transcarioca_raw['x'].apply(lambda x: get_velocidade(x))
df_transcarioca['CGCT'] = df_transcarioca_raw['x'].apply(lambda x:x[108:111])
df_transcarioca['LATITUDE'] = df_transcarioca_raw['x'].apply(lambda x: get_latitude(x))
df_transcarioca['LONGITUDE'] = df_transcarioca_raw['x'].apply(lambda x: get_longitude(x))

# Salvando em csv
df_transcarioca.to_csv('amostra_transcarioca_tratada.csv', sep=';', index=False)
df_transcarioca.head(20)

NameError: name 'df_transcarioca_raw' is not defined

# SPLICE_007201820190205.txt_amostra.txt

In [9]:
# Nome para colunas desconhecidas
colnames = ['x']
# Usando separador '|' apenas para conseguir importar e depois tentar tratar
df = pd.read_csv('SPLICE_0072018_20190205_.txt_amostra.txt', sep='|', names = colnames)
# df_splice_raw
# Separando as colunas manualmente

# df_splice_split = separar_colunas_df(df_splice_raw)

# def separar_colunas_df(df):
lista_alfabeto = list(string.ascii_lowercase)

df.iloc[:,0] = df.iloc[:,0].replace({'( ){2,} ':'|'},regex=True)

lista_e = []
df.iloc[:,0] = df.iloc[:,0].apply(lambda x: correctSplit(x))
for i in range(len(df)):
    lista_e.append(len(df.iloc[:,0][i].split('|'))==4)
#     print(len(df.iloc[:,0][i].split('|')))

# df.iloc[3][0][78:]

a = df.x.where(lista_e)
print(a.dropna().iloc[0])
print(len(a.dropna().iloc[0]))
print(df.iloc[0][0])
print(len(df.iloc[0][0]))
# print(df.shape)
# print(len(lista_e))
# print(df.iloc[:,0][0].split('|'))
# for i in range (len(df.iloc[:,0][0].split('|'))):
#     try:
#         df[lista_alfabeto[i]] = df.iloc[:,0].apply(lambda x: x.split('|')[i])
#     except:
#         df[lista_alfabeto[i]] = 'erro'
# df.drop(df.columns[0], axis = 1, inplace = True)

# df

# df_splice['x'] = df_splice.x.replace({'( ){2,} ':'|'},regex=True) # Substituindo 2 ou + espaços pelo caracter "|"
# df_splice['x'] = df_splice.x.apply(lambda x: correctSplit(x))
# df_splice['veiculo'] = df_splice.x.apply(lambda x: x.split('|')[0])
# df_splice['a'] = df_splice.x.apply(lambda x: x.split('|')[1])
# df_splice['b'] = df_splice.x.apply(lambda x: x.split('|')[2])
# df_splice['c'] = df_splice.x.apply(lambda x: x.split('|')[3])

# Dropando a coluna x utilizada para separar as outras colunas
# df_splice.drop('x', axis = 1, inplace = True)

# df_splice_split.sample(5)

FileNotFoundError: [Errno 2] No such file or directory: 'SPLICE_0072018_20190205_.txt_amostra.txt'

In [10]:
i = 0
print(df.iloc[i][0][78:], end = '\n\n')
print(df.iloc[i][0][30:], end = '\n\n')
print(df.iloc[i][0][140:], end = '\n\n')

NameError: name 'df' is not defined

# FISCALTECH_0042016_06_2019.txt_amostra.txt

In [11]:
# Nome para colunas desconhecidas
colnames = ['x']
# Usando separador '|' apenas para conseguir importar e depois tentar tratar
df_fiscaltech_raw = pd.read_csv('FISCALTECH_0042016_06_2019.txt_amostra.txt', sep='|', names = colnames)
df_fiscaltech = separar_colunas_df(df_fiscaltech_raw)


# Extraindo partes onde possui informação requisitada
df_fiscaltech['b0'] = df_fiscaltech.b
df_fiscaltech['d'] = df_fiscaltech.c # aqui existe latidude e longitude, além de um número prévio que também poderia sar o 
#código do tipo
df_fiscaltech['b'] = df_fiscaltech.b0.apply(lambda x: x.split(' ')[0])
df_fiscaltech['c'] = df_fiscaltech.b0.apply(lambda x: x.split(' ')[2][:17])

df_fiscaltech.head(4)

,a,b,c,b0,d
0,1,0106201903270901800FSCII2175,04200611130042016,0106201903270901800FSCII2175 0420061113004201...,3-23000409-43334161
1,1,0106201901164305100FSCII2214,04201712110042016,0106201901164305100FSCII2214 0420171211004201...,1
2,1,0106201901120105000FSCII2174,04200711240042016,0106201901120105000FSCII2174 0420071124004201...,4-23001615-43328645
3,1,0106201903304702100FSCII2175,04200611140042016,0106201903304702100FSCII2175 0420061114004201...,4-23000409-43334161


# Tratamento FISCALTECH

In [12]:
def get_datahora(string):
    date = string[:13]
    try: 
        date = datetime.strptime(date, "%d%m%Y%H%M%S")
    except:
        date = 'erro'
    return date


def get_lat(string):
    if len(string.split('-')) == 1: #se estamos no brasil, Lat e Long são <0. Então esses campos estão preenchidos, há '-'
        return ''
    else:
        return '-'+string.split('-')[1][:2]+'.'+string.split('-')[1][2:]

def get_long(string):
    if len(string.split('-')) == 1: #se estamos no brasil, Lat e Long são <0. Então esses campos estão preenchidos, há '-'
        return ''
    else:
        return '-'+string.split('-')[2][:2]+'.'+string.split('-')[2][2:]

def get_placa(string):
    if len(string)==1:
        return ''
    else:
        return string[:7]

def get_tipo(string):
    if len(string)==1:
        return string
    else:
        return string[7:8]


#Separação de colunas 

df_fiscaltech['PLACA'] = df_fiscaltech.a.apply(lambda x: get_placa(x))
df_fiscaltech['TIPO'] = df_fiscaltech.a.apply(lambda x: get_tipo(x))
df_fiscaltech['CODCET'] = df_fiscaltech.c.apply(lambda x: x[:10])
df_fiscaltech['CGT'] = df_fiscaltech.c.apply(lambda x: x[:3])
df_fiscaltech['DATAHORA'] = df_fiscaltech.b.apply(lambda x: get_datahora(x))
df_fiscaltech['VELOCIDADE'] = df_fiscaltech.b.apply(lambda x: x[15:17]) #o certo seria [14:19], mas estamos padronizando
#sem casa decimal
df_fiscaltech['LATITUDE'] = df_fiscaltech.d.apply(lambda x: get_lat(x))
df_fiscaltech['LONGITUDE'] = df_fiscaltech.d.apply(lambda x: get_long(x)) #perceber que deixo um número antes do - de fora

df_fiscaltech.drop(['a','b','c','d','b0'], axis = 1, inplace = True)

# Salvando em csv
df_fiscaltech.to_csv('amostra_fiscaltech_tratada.csv', sep=';', index=False)

# Print
df_fiscaltech.sample(10)

,PLACA,TIPO,CODCET,CGT,DATAHORA,VELOCIDADE,LATITUDE,LONGITUDE
77,,1,0420021123,042,2019-06-01 05:12:05,55,-23.000226,-43.353781
51,,1,0420061112,042,2019-06-01 01:15:00,02,-23.000409,-43.334161
78,,1,0420181112,042,2019-06-01 01:13:00,56,-22.973304,-43.39867
42,,1,0420121221,042,2019-06-01 05:13:04,77,-22.966068,-43.396717
29,,1,0420071113,042,2019-06-01 03:39:01,69,-23.00141,-43.328091
92,,1,0420071124,042,2019-06-01 03:40:04,63,-23.001615,-43.328645
79,,1,0420041123,042,2019-06-01 01:13:04,74,-23.000132,-43.33874
15,,1,0420081112,042,2019-06-01 01:17:02,55,-23.00303,-43.32414
76,,1,0420081112,042,2019-06-01 06:19:02,15,-23.00303,-43.32414
52,,1,0420061114,042,2019-06-01 01:15:01,27,-23.000409,-43.334161
